<h3>Pre-requisites.</h3>    

- pip install ffmpeg-python 
- pip install probe    
- Download ffmpeg from here -> https://www.gyan.dev/ffmpeg/builds/    
- Place the downloaded ffmpeg inside the Python folder where the python.exe resides. (Assuming you use windows, I am not sure of what do with mac, sorry :-( )


In [12]:
#Importing the necessary headers
import os
import subprocess
import ffmpeg
import cv2

<h3>All your Declarations go here.</h3>

In [11]:
#Defining the root media input and output directory.
rootmediadir = r'D:\University_of_Leicester\GitHub_Projects\FruitpunchAI\input'
outputdir = r'D:\University_of_Leicester\GitHub_Projects\FruitpunchAI\output'

#resolutionMultiplier can be used to multiply the resolution of the height and width of the output file.
resolutionMultiplier = 1

#Format specifier, please use only "mpg" /  "mp4" as the input.
outputFormat = "mp4"

#BitrateMultiplier is used to multiply the pre-calculated minimum best bitrate.
BitrateMultiplier = 3


<h3>The code does its stuff from here.</h3>

In [14]:
def videoDetails(filename):
    
    #The imported cv2 is utilizied here.
    video = cv2.VideoCapture(filename)

    #Fetching the frame count.
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    
    #Fetching the fps.
    fps = video.get(cv2.CAP_PROP_FPS)
    
    #Fetching the width of the video as a float.
    width = video.get(3)
    
    #Fetching the height of the video as a float.
    height = video.get(4)

    #Returning the duration, height and width of the video.
    return frame_count / fps , height, width

In [15]:
def compress_video(videoPath, ouputFileName, TargetSize, resolutionVideo):
    
    # The Reference to encoding bitrates : https://en.wikipedia.org/wiki/Bit_rate#Encoding_bit_rate
    # The following is the min and max audio bitrate, choose as required.
    minAudioBitrate = 8000
    maxAudioBitrate = 16000
    probe = ffmpeg.probe(videoPath)
    
    # The Video duration is measured in seconds.
    duration = float(probe['format']['duration'])
    
    # The Audio bitrate is measured in in bps.
    AudioBitrate = float(next((s for s in probe['streams'] if s['codec_type'] == 'audio'), None)['bit_rate'])
    
    # Target total bitrate is obtained by the following formula in bps.
    targetTotalBitrate = (TargetSize * 1024 * 8) / (1.073741824 * duration)

    # Target audio bitrate is measured in bps.
    if 10 * AudioBitrate > targetTotalBitrate:
        AudioBitrate = targetTotalBitrate / 10
        if AudioBitrate < minAudioBitrate < targetTotalBitrate:
            AudioBitrate = minAudioBitrate
        elif AudioBitrate > maxAudioBitrate:
            AudioBitrate = maxAudioBitrate
            
    # Target video bitrate, in bps.
    videoBitrate = targetTotalBitrate - AudioBitrate

    i = ffmpeg.input(videoPath)
    
    if outputFormat == 'mp4':    
        #First pass
        ffmpeg.output(i, '/dev/null' if os.path.exists('/dev/null') else 'NUL',
                      **{'c:v': 'libx264', 'b:v': videoBitrate, 'pass': 1, 'f': 'mp4' , 's' : resolutionVideo}
                      ).overwrite_output().run()
        #Second pass to improve quality of output.
        ffmpeg.output(i, ouputFileName,
                      **{'c:v': 'libx264', 'b:v': videoBitrate, 'pass': 2 , 's' : resolutionVideo}
                      ).overwrite_output().run()
        
    elif outputFormat == 'mpg':    
        #First pass
        ffmpeg.output(i, '/dev/null' if os.path.exists('/dev/null') else 'NUL',
                      **{'c:v': 'libx264', 'b:v': videoBitrate, 'pass': 1, 'f': 'MPEG' , 's' : resolutionVideo}
                      ).overwrite_output().run()
        #Second pass to improve quality of output.
        ffmpeg.output(i, ouputFileName,
                      **{'c:v': 'libx264', 'b:v': videoBitrate, 'pass': 2 , 's' : resolutionVideo}
                      ).overwrite_output().run()
        
    else:
        print('ERROR - Please provide the outputFormat as either mp4 or mpg, sorry about the inconvenience. :-) ')

In [16]:
%%time

#Obtaining the files and converting them.
#Fetching all directories and subdirectories in rootmediadir.
for subdirs, dirs, files in os.walk(rootmediadir):
    
    #Fetching each file.
    for file in files:
        
        media_in = subdirs + "/" + file
        
        print('\n___________________________________')
        print('Processing NOW - ', file)
        
        #Obtaining the duration, height and width of the video file.
        duration, height, width = videoDetails(media_in)
        
        best_min_size = ((32000 + 100000) * (1.073741824 * duration) / (8 * 1024)) * BitrateMultiplier
        
        resolutionVideo = str(int(width)*resolutionMultiplier) + 'x' + str(int(height)*resolutionMultiplier)
        filename = os.path.splitext(file)[0]
        compress_video(media_in, outputdir + '/' + 'output_' + filename + '.' + outputFormat, best_min_size, resolutionVideo)
        
        print('Processing COMPLETED - ', 'output_' + filename + '.' + outputFormat)
        print('___________________________________')

#Printing the output directory.
print('\nThe converted output files can be found in:\n', outputdir ,'\n')


___________________________________
Processing NOW -  100k_Mario1.mp4
ERROR - Please provide the outputFormat as either mp4 or mpg, sorry about the inconvenience. :-) 
Processing COMPLETED -  output_100k_Mario1.gif
___________________________________

___________________________________
Processing NOW -  100k_Mario2.mp4
ERROR - Please provide the outputFormat as either mp4 or mpg, sorry about the inconvenience. :-) 
Processing COMPLETED -  output_100k_Mario2.gif
___________________________________

___________________________________
Processing NOW -  150k_Mario1.mp4
ERROR - Please provide the outputFormat as either mp4 or mpg, sorry about the inconvenience. :-) 
Processing COMPLETED -  output_150k_Mario1.gif
___________________________________

___________________________________
Processing NOW -  20k_Mario1.mp4
ERROR - Please provide the outputFormat as either mp4 or mpg, sorry about the inconvenience. :-) 
Processing COMPLETED -  output_20k_Mario1.gif
______________________________

ZeroDivisionError: float division by zero